# Separador de sentenças do Eproc SC

In [1]:
import PyPDF2
import os

## Configuração dos Diretórios de Entrada e Saída

* O diretório de entrada é o que possui os arquivos PDFs com várias sentenças, e apenas isso
* O diretório de saída será onde as sentenças separadas serão armazenadas, um PDF por sentença

In [7]:
input_folder = "input"

print('Arquivos para leitura no diretório de entrada:')
input_files = os.listdir(input_folder)
input_files

Arquivos para leitura no diretório de entrada:


['SentencasExemplo.pdf']

In [8]:
# Diretório de saída, onde as sentenças separadas serão armazenadas
output_folder = "output"

# Verificar se o diretório não existe
if not os.path.exists(output_folder):
    # Criar o diretório de saída se não existir
    os.makedirs(output_folder)
    print(f"Diretório de saída: `{output_folder}` criado com sucesso!")
else:
    # Se existir, só avisa que já existe
    print(f"O diretório de saída `{output_folder}` já existe!")


Diretório de saída: `output` criado com sucesso!


## Processamento de arquivo

**Sentença de teste**
1. Páginas: \[1,5\] -> 0 ,4
2. Pgs: \[6-10\]    -> 5 ,9
3. Pgs: \[11-16\]   -> 10,15
4. Pgs: \[17-25\]   -> 16,24
5. Pgs: \[26-30\]   -> 25,29

In [9]:
# Função que lê um arquivo PDF (pdf_reader já aberto) e divide em múltiplos PDFs
# de acordo com a lista de páginas (pages) marcadas.
# O nome dos arquivos gerados são os números dos processos (sentence_ids)

def split_sentence_files(pdf_reader,pages,sentence_ids):
    # Marcador da primeira página 
    start_page = 0
    for doc_index, end_page in enumerate(pages):
        sentence_id = sentence_ids[doc_index]
        output_filename = f"{output_folder}/{sentence_id}.pdf"
        
        print('- Em processamento: [',doc_index+1,"/",len(pages),"] : [", start_page, ",", end_page , '] - ', sentence_ids[doc_index], ":",output_filename)
        # Escreve arquivos na saída
        pdf_writer = PyPDF2.PdfWriter()
        pdf_writer.append(fileobj=pdf_reader, pages=(start_page, end_page))
        output = open(output_filename, "wb")
        pdf_writer.write(output)
        pdf_writer.close()
        output.close()
        
        start_page = end_page

In [11]:
# Processamento em lote de todos os arquivos no diretório de entrada!
# Para cada arquivo no diretório de entrada, divida-os em múltiplos no diretório de saída!

for file in input_files:
    filename = f'{input_folder}/{file}'
    print(f'>>>>> Processando arquivo: `{filename}`')
    # Ler arquivo de entrada
    pdf_fileobj = open(filename,'rb')
    
    # Criar o leitor do arquivo de entrada
    pdf_reader = PyPDF2.PdfReader(pdf_fileobj)
    
    # Lista com as marcações das páginas de início e final de sentença
    pages_divisors = [] # Vetor com os números das páginas que iniciam cada sentença
    sentence_ids = []   # Vetor com o Identificador da sentença que é o número do processo
    for i in range(len(pdf_reader.pages)): # Para cada página do arquivo aberto
        page_obj=pdf_reader.pages[i]       # Lê a página
        page_text=page_obj.extract_text()  # Extrai o texto
        page_lines = page_text.split('\n') # Quebra as linhas
        page_first_line = page_lines[0]    # Obtém o texto da primeira linha
        
        # O que identifica a primeira página de cada sentença é o endereço da UFSC (baseado no arquivo de teste)
        if page_first_line == 'Av. Des. Vitor Lima, 183, fundos- Campus da UFSC - Bairro: Serrinha - CEP: 88040-400 - Fone: (48)3287-5019 - Email:':
            pages_divisors.append(i)                         # Marca a página de início da sentença
            sentence_ids.append(page_lines[-4].split()[0])   # Obtém o identificador da sentença (número do processo) que está no rodapé da página
    
    pages_divisors.pop(0) # Remove o marcador de página 0, pois página que inícia a primeira sentença sempre será a primeira
    pages_divisors.append(len(pdf_reader.pages)) # Adiciona a última página do arquivo na lista de páginas.

    print(f'## Páginas que dividem as sentenças no arquivo `{file}`: {pages_divisors}')
    print(f'## Número dos processos do arquivo `{file}`: {sentence_ids}')
    split_sentence_files(pdf_reader,pages_divisors,sentence_ids) # Processa o arquivo

    pdf_reader.close()
    pdf_fileobj.close()
    print()
    print()

>>>>> Processando arquivo: `input/SentencasExemplo.pdf`
## Páginas que dividem as sentenças no arquivo `SentencasExemplo.pdf`: [5, 10, 16, 25, 30]
## Número dos processos do arquivo `SentencasExemplo.pdf`: ['5027803-79.2022.8.24.0090', '5028771-75.2023.8.24.0090', '5126667-62.2022.8.24.0023', '5022213-87.2023.8.24.0090', '5011176-63.2023.8.24.0090']
- Em processamento: [ 1 / 5 ] : [ 0 , 5 ] -  5027803-79.2022.8.24.0090 : output/5027803-79.2022.8.24.0090.pdf
- Em processamento: [ 2 / 5 ] : [ 5 , 10 ] -  5028771-75.2023.8.24.0090 : output/5028771-75.2023.8.24.0090.pdf
- Em processamento: [ 3 / 5 ] : [ 10 , 16 ] -  5126667-62.2022.8.24.0023 : output/5126667-62.2022.8.24.0023.pdf
- Em processamento: [ 4 / 5 ] : [ 16 , 25 ] -  5022213-87.2023.8.24.0090 : output/5022213-87.2023.8.24.0090.pdf
- Em processamento: [ 5 / 5 ] : [ 25 , 30 ] -  5011176-63.2023.8.24.0090 : output/5011176-63.2023.8.24.0090.pdf


